# The Outline

1. Using the pre-trained available BART-ParaDetox model, we run all of the comments in Jigsaw dataset through it to generate its parallel detoxed dataset.
2. Using the generated parallel dataset, we use it to train our own BART model
3. The trained BART model will then be used to generate the detoxified sentence of any input sentence.

## Setup the python environment
Ideally you run the following cells with the virtual environment created using python3 (anything before 3.10 version works from our testing) with the packages specified in the `requirements.txt`.

In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt


Read and load the Jigsaw dataset with the relevant columns:
- `comment_text` : the column containing the input toxic sentence
- `toxicity` : the column cotaining the toxicity score of the input toxic sentence

## 1. Generating the parallel dataset from Jigsaw

In [ ]:
df = pd.read_csv('all_data.csv')
target_columns = ['comment_text', 'toxicity', 'severe_toxicity']

### Data Cleaning

In [ ]:
import re
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'http\S+', '', text) # Remove URLs
        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
        text = text.lower()
        # Handle self-censored words and emojis as needed
        pass
    
    else:
        return ""
    return text

# cleaned = df['comment_text'].apply(clean_text)
# cleaned

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("s-nlp/bart-base-detox")
model = AutoModelForSeq2SeqLM.from_pretrained("s-nlp/bart-base-detox")
# 'that sick fuck is going to be out in 54 years.', 
toxics = ["Hope they're both beaten to death in the can. Should have gotten the death penalty. Says everything about Kansas social services. It's a sick country."]
tokens = tokenizer(toxics, return_tensors='pt', padding=True)
tokens = model.generate(**tokens, num_return_sequences=1, do_sample=False,
                        temperature=1.0, repetition_penalty=10.0,
                        max_length=128, num_beams=5)
neutrals = tokenizer.decode(tokens[0, ...], skip_special_tokens=True)
print(neutrals) # stdout: She is going to be out in 54 years.


Hope they're both beaten to death in the can. Should have gotten the death penalty. Says everything about Kansas social services. It's a sick country.
